<a href="https://colab.research.google.com/github/Lakshya8600/stockmarket_ML__XGB_CNN/blob/main/hybrid_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install transformers

import pandas as pd
import numpy as np
from collections import defaultdict
import itertools
from transformers import pipeline
import csv
import requests
import re
import ast

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch


In [2]:
# Load FinBERT sentiment model
# You need to install either of Tensorflow or PyTorch to run FinBERT

finbert = pipeline("text-classification", model="ProsusAI/finbert", return_all_scores=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [25]:
# Importing the dataset as pandas dataframe:-


# url = "https://raw.githubusercontent.com/Lakshya8600/stockmarket_ML__XGB_CNN/main/data/20230714.csv" ##Try with this url if facing issue
url = "https://raw.githubusercontent.com/Lakshya8600/stockmarket_ML__XGB_CNN/main/data/20230714.csv"

response = requests.get(url)
data = response.text.splitlines()  # Split into lines

# Convert each line from string to list
headlines = [ast.literal_eval(line) for line in data]  # Safely convert string list to actual list

# Create a DataFrame
df_true = pd.DataFrame({"headlines": headlines})

# Print first 5 rows
print(df_true.head())


                                           headlines
0  [india, reimposes, windfall, tax, on, petroleu...
1  [arsenal, signs, jurrien, timber, shirt, numbe...
2  [who, is, kevin, spacey, all, about, the, osca...
3  [ex, manchester, city, footballer, cleared, of...
4  [nyse, suspends, trading, in, azure, global, d...


In [26]:
df = df_true.copy(deep=True)


In [30]:
df["headline_text"] = df["headlines"].apply(lambda x: " ".join(x))  # Join list elements with a space, i.e makes Full String
df.head()

# Now our df contains headline_text(string) and headlines(list of words) also

,headlines,headline_text
0,"[india, reimposes, windfall, tax, on, petroleu...",india reimposes windfall tax on petroleum crud...
1,"[arsenal, signs, jurrien, timber, shirt, numbe...",arsenal signs jurrien timber shirt number insp...
2,"[who, is, kevin, spacey, all, about, the, osca...",who is kevin spacey all about the oscar winnin...
3,"[ex, manchester, city, footballer, cleared, of...",ex manchester city footballer cleared of rape ...
4,"[nyse, suspends, trading, in, azure, global, d...",nyse suspends trading in azure global delistin...


In [33]:
# Load FinBERT model & tokenizer
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


In [34]:
# function to extract sentiment probabilities:-

from scipy.special import softmax

def get_sentiment_probs(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs).logits
    probs = softmax(outputs.numpy())  # Convert logits to probabilities
    return probs[0]  # Return array of [negative, neutral, positive] probabilities


In [36]:
df["sentiment_probs"] = df["headline_text"].apply(get_sentiment_probs)

# Convert list output into separate columns
df["negative"] = df["sentiment_probs"].apply(lambda x: x[0])
df["neutral"] = df["sentiment_probs"].apply(lambda x: x[1])
df["positive"] = df["sentiment_probs"].apply(lambda x: x[2])

# Drop the raw probability column
df.drop(columns=["sentiment_probs"], inplace=True)

print(df.head(20))  # Check new features


                                            headlines  \
0   [india, reimposes, windfall, tax, on, petroleu...   
1   [arsenal, signs, jurrien, timber, shirt, numbe...   
2   [who, is, kevin, spacey, all, about, the, osca...   
3   [ex, manchester, city, footballer, cleared, of...   
4   [nyse, suspends, trading, in, azure, global, d...   
5   [corporate, affairs, ministry, to, withdraw, 7...   
6   [bbc, proms, 2023, schedule, filled, with, div...   
7   [stock, radar, 15, rally, in, a, month, this, ...   
8   [how, founder, friendly, culture, could, hurt,...   
9   [gujarat, govt, announces, rs, 240, cr, relief...   
10  [netweb, technologies, mobilises, rs, 189, cr,...   
11  [pm, modi, french, assembly, president, discus...   
12  [strike, at, gatwick, airport, in, south, lond...   
13  [mission, impossible, dead, reckoning, part, o...   
14  [passenger, in, trouble, for, opening, cover, ...   
15  [yamuna, water, enters, gandhi, memorial, at, ...   
16  [deadpool, 3, hit, by, sag,